![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Construcción e implementación de modelos basados en Boosting
En este notebook aprenderá a construir e implementar modelos de Adaboost, Gradient Boosting y XGBoost. El primer modelo lo desarrollará de forma manual y usando la librería especializada sklearn, mientras que los otros dos los desarrollará solamente usando la librería.

## Instrucciones Generales:

Los modelos que construirá por medio de este notebook deberán predecir si un usuario deja o no de usar los servicios de una compañía (churn) teniendo en cuenta diferentes variables. Para conocer más detalles de la base de 'churn' puede ingresar al siguiente vínculo: http://srepho.github.io/Churn/Churn

Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Importar base de datos y librerías
import pandas as pd
import numpy as np

# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/churn.csv')

In [10]:
# Selección de variables numéricas (X)
X = data.iloc[:, [1,2,6,7,8,9,10]].astype(float)
# Tranformación de variables booleanas a floats
X = X.join((data.iloc[:, [4,5]] == 'no').astype(float))

# Definición variable de interés binaria (y)
y = (data.iloc[:, -1] == 'True.').astype(int)

In [11]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)
n_samples = X_train.shape[0]

## Adaboost manual

In [12]:
# Definición de la cantidad de árboles de decisión del modelo
n_estimators = 10

# Definición de DataFrame para guardar pesos de las observaciones en cada árbol de decisión
weights = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

# Asignación los mismos pesos para todas las observaciones en el árbol 0
t = 0
weights[t] = 1 / n_samples

# Visualización de DataFrame de pesos
weights.head()

,0,1,2,3,4,5,6,7,8,9
2953,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2510,0.000448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Definición y entrenamiento (fit) del primer árbol de decisión (DecisionTreeClassifier)
trees = []
trees.append(DecisionTreeClassifier(max_depth=1))
trees[t].fit(X_train, y_train, sample_weight=weights[t].values)

DecisionTreeClassifier(max_depth=1)

In [14]:
# Estimación del error del primer árbol de decisión
y_pred_ = trees[t].predict(X_train)
error = []
error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, weights[t].values))
error[t]

TypeError: too many positional arguments

In [ ]:
# Cálculo del factor alpha del primer árbol de decisión
alpha = []
alpha.append(np.log((1 - error[t]) / error[t])/2)
alpha[t]

In [ ]:
# Actualización de los pesos a considerar en el segundo árbol (t+1)
weights[t + 1] = weights[t]
filter_ = y_pred_ != y_train

weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])

In [ ]:
# Normalización de los pesos
weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [ ]:
# Visualización de DataFrame de pesos
weights.head()

In [ ]:
# Definición de loop que itera sobre todos los árboles definidos realizando los mismos cáculos presentados anteriormente
for t in range(1, n_estimators):
    # Definición y entrenamiento (fit) del árbol t
    trees.append(DecisionTreeClassifier(max_depth=1))
    trees[t].fit(X_train, y_train, sample_weight=weights[t].values)
    y_pred_ = trees[t].predict(X_train)
    # Estimación del error del árbol t
    error.append(1 - metrics.balanced_accuracy_score(y_pred_, y_train, weights[t].values))
    # Cálculo del factor alpha para el árbol t
    alpha.append(np.log((1 - error[t]) / error[t]) / 2)
    # Actualización de pesos para el árbol t+2
    weights[t + 1] = weights[t]
    filter_ = y_pred_ != y_train
    weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])
    weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [ ]:
# Visualización de los errores de cada árbol
error

In [ ]:
# Visualización de DataFrame de pesos
weights.head()

In [ ]:
# Solo se usan modelos con error menor a 0.5
new_n_estimators = np.sum([x<0.5 for x in error])

In [ ]:
# Prección sobre la muestra de test
y_pred_all = np.zeros((X_test.shape[0], new_n_estimators))
for t in range(new_n_estimators):
    y_pred_all[:, t] = trees[t].predict(X_test)

In [ ]:
# Obtención de la predicción final al poderar las predicciones por el factor aplha
y_pred = (np.sum(y_pred_all * alpha[:new_n_estimators], axis=1) >= 1).astype(np.int)

In [ ]:
# Impresión del desempeño modelo
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

In [ ]:
# Definición de un sólo árbol de decisón para compara el desempeño
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

## Adaboost usando sklearn

In [15]:
# Importación y definición de modelo AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf

AdaBoostClassifier()

In [16]:
# Entrenamiento (fit) y desempeño del modelo AdaBoostClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.36771300448430494, 0.8718181818181818)

## Gradient Boosting usando sklearn

In [17]:
# Importación y definición de modelo GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf

GradientBoostingClassifier()

In [18]:
# Entrenamiento (fit) y desempeño del modelo GradientBoostingClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4644549763033175, 0.8972727272727272)

## XGBoost usando sklearn
Instalar la librería xgboost usando el comando: pip install xgboost

In [20]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/03/e6/4aef6799badc2693548559bad5b56d56cfe89eada337c815fdfe92175250/xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.3 MB/s eta 0:00:00a 0:00:01


In [21]:
# Importación y definición de modelo XGBClassifier

from xgboost import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [22]:
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.4521739130434782, 0.8854545454545455)